In [ ]:
from IPython.display import display, Math
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

# Eigenvectors of the stiffness matrix in limit state problems
[Dominik Kern](https://orcid.org/0000-0002-1958-2982),
[Chiara Hergl](https://orcid.org/0000-0002-4016-9113),
[Thomas Nagel](https://orcid.org/0000-0001-8459-4616)
# Minimal Example of Beam Buckling with Bending only
In this formulation the axial force $F$ is a parameter in the stiffness matrix $\mathbf{C}$.

![Euler buckling case 2](euler_buckling.svg)
**TODO** remove $u$ from figure

Non-dimensional potential energy for beam bending (transversal displacement $w$) with axial force $F$ and stabilizing spring (stiffness $c$)    
$\Pi = \frac{1}{2}\int_0^1 w''(x)^2 -F w'^2 \mathrm{d}x +\frac{1}{2}cw\left(\frac{1}{2}\right)^2$

with following relations to the dimensional variables 
- $F = \dfrac{F_\mathrm{dim}}{F_\mathrm{ref}}$,
- $c = \dfrac{c_\mathrm{dim}l}{F_\mathrm{ref}}$
with $F_\mathrm{ref} = \dfrac{EI}{l^2}$.

Ritz' method with buckling mode shapes $w_k=\sin\bigl(k\pi x \bigr)$ leads to $k$th buckling load 
$F_k = (k\pi)^2+\frac{2c}{(k\pi)^2}\sin^2\left(\frac{k\pi}{2}\right)$. 

## Symbolic calculations

Apply Ritz discretization with two DoF and evaluate first variation of potential energy

In [ ]:
x, X, k, c, F, W1, W2, dW1, dW2  = sp.symbols('x X k c F W1 W2 dW1 dW2')

In [ ]:
w = sp.sin(k*sp.pi*x) 

W = W1*w.subs(k,1) + W2*w.subs(k,2)
Wx = W.diff(x)
Wxx = Wx.diff(x)

dW = dW1*w.subs(k,1) + dW2*w.subs(k,2)
dWx = dW.diff(x)
dWxx = dWx.diff(x)

dPi = sp.integrate(Wxx*dWxx - F*Wx*dWx, (x,0,1) ) + c*W.subs(x,X)*dW.subs(x,X)
display(dPi)

Generalized forces

In [ ]:
Q1 = sp.simplify(dPi.diff(W1))
display(Math(f'Q_1 = {sp.latex(Q1)}'))
Q2 = sp.simplify(dPi.diff(W2))
display(Math(f'Q_2 = {sp.latex(Q2)}'))

Stiffness matrix (symmetric, remember $\sin(2x)=2\sin(x)\cos(x)$ )

In [ ]:
C11 = sp.expand(Q1).coeff(dW1, 1)
display(C11)
C12 = sp.expand(Q1).coeff(dW2, 1)
display(C12)
C21 = sp.expand(Q2).coeff(dW1, 1)
display(C21)
C22 = sp.expand(Q2).coeff(dW2, 1)
display(C22)

C = sp.Matrix(((C11, C12), (C21, C22)))
display(Math(f'C = {sp.latex(C)}'))

Buckling occurs when determinant of stiffness matrix vanishes

In [ ]:
char_eq = sp.simplify(sp.Determinant(C).doit())
buckling_load = sp.solve(char_eq, F)
F1 = sp.simplify(buckling_load[0])
F2 = sp.simplify(buckling_load[1])
display(Math(f'F_1 = {sp.latex(F1)}'))
display(Math(f'F_2 = {sp.latex(F2)}'))

Eigenvector from first line (same result from second)

In [ ]:
Cabs = sp.simplify(sp.sqrt(C11**2 + C12**2)) 

w1_ev1   = sp.simplify( C12.subs(F, F1))
w2_ev1   = sp.simplify(-C11.subs(F, F1))
wabs_ev1 = sp.simplify(Cabs.subs(F, F1))

w1_ev2   = sp.simplify( C12.subs(F, F2))
w2_ev2   = sp.simplify(-C11.subs(F, F2))
wabs_ev2 = sp.simplify(Cabs.subs(F, F2))

# same result from second line
#Cabs = sp.simplify(sp.sqrt(C21**2 + C22**2)) 

#w1_ev1   = sp.simplify( C22.subs(F, F1))
#w2_ev1   = sp.simplify(-C21.subs(F, F1))
#wabs_ev1 = sp.simplify(Cabs.subs(F, F1))

#w1_ev2   = sp.simplify( C22.subs(F, F2))
#w2_ev2   = sp.simplify(-C21.subs(F, F2))
#wabs_ev2 = sp.simplify(Cabs.subs(F, F2))

In [ ]:
w1_ev1_num   = sp.lambdify((X, c), w1_ev1, modules='numpy') 
w2_ev1_num   = sp.lambdify((X, c), w2_ev1, modules='numpy') 
wabs_ev1_num = sp.lambdify((X, c), wabs_ev1, modules='numpy') 
w1_ev2_num   = sp.lambdify((X, c), w1_ev2, modules='numpy') 
w2_ev2_num   = sp.lambdify((X, c), w2_ev2, modules='numpy') 
wabs_ev2_num = sp.lambdify((X, c), wabs_ev2, modules='numpy') 

Evaluate discriminant to find spring stiffness when both buckling loads share the same value

In [ ]:
char_eqF = char_eq.diff(F)
F_ex = sp.solve(char_eqF,F)
char_eq_ex = sp.simplify(char_eq.subs(F,F_ex[0]))
display(Math(f'0 = {sp.latex(char_eq_ex)}'))
c_double = sp.solve(char_eq_ex, c)
cm = sp.simplify(c_double[0])
cp = sp.simplify(c_double[1])
display(Math(f'c_m = {sp.latex(cm)}'))
display(Math(f'c_p = {sp.latex(cp)}'))

## Numerical evaluations

In [ ]:
c_grid = np.linspace(1, 300)   # for c=0 one of the eigenvectors reduces to zero

### stabilizing spring in the middle ($X=\frac{1}{2}$) _symmetric case_

In [ ]:
Xsym = 0.5
F1_sym = sp.lambdify((c), F1.subs(X, Xsym), modules='numpy')
F2_sym = sp.lambdify((c), F2.subs(X, Xsym), modules='numpy')

In [ ]:
plt.plot(c_grid,F1_sym(c_grid), c_grid,F2_sym(c_grid))
plt.xlabel("stiffness");
plt.ylabel("buckling load");
plt.legend(["F_1", "F_2"]);

Note that the horizontal line, independent on color, corresponds to the second mode shape and the sloped line to the first mode shape. The Physics behind this plot are that, the spring only stabilizes against first mode buckling and is ineffective for the second mode (node point).

There is one value of the spring, when both buckling loads are equal.

In [ ]:
char_eq_sym = char_eq_ex.subs(X, Xsym)
display(char_eq_sym)
char_eq_sym_num = sp.lambdify((c), char_eq_sym, modules='numpy')
#plt.plot(c_grid, char_eq_sym_num(c_grid));

In [ ]:
cm_sym = cm.subs(X, Xsym).evalf()
display(Math(f'c_m = {sp.latex(cm_sym)}'))
cp_sym = cp.subs(X, Xsym).evalf()
display(Math(f'c_p = {sp.latex(cp_sym)}'))

eigenvectors show a jump: below the critical spring stiffness the lower eigenvalue corresponds to the first mode the higher eigenvalue to the second mode, above the critical value it is vice versa.

### stabilizing spring somewhere ($X\ne\frac{1}{2}$) _asymmetric case_

In [ ]:
Xasym = 0.51
F1_asym = sp.lambdify((c), F1.subs(X, Xasym), modules='numpy')
F2_asym = sp.lambdify((c), F2.subs(X, Xasym), modules='numpy')

In [ ]:
plt.plot(c_grid,F1_asym(c_grid),'b', c_grid,F2_asym(c_grid),'r')
plt.xlabel("stiffness");
plt.ylabel("buckling load");
plt.legend(["F_1", "F_2"]);

There is no common value of both buckling modes for any spring stiffness, apparent from the plot, as well as from the numerical expressions.

In [ ]:
cm_asym = cm.subs(X, Xasym).evalf()
display(Math(f'c_m = {sp.latex(cm_asym)}'))
cp_asym = cp.subs(X, Xasym).evalf()
display(Math(f'c_p = {sp.latex(cp_asym)}'))

#### normalized Eigenvectors

In [ ]:
plt.plot(c_grid, np.abs(w1_ev1_num(Xasym, c_grid)/wabs_ev1_num(Xasym, c_grid)),'b', c_grid, np.abs(w2_ev1_num(Xasym, c_grid)/wabs_ev1_num(Xasym, c_grid)),'b--');
plt.xlabel('$c$');
plt.legend(['$w_1$', '$w_2$']);

For the lower buckling load there is a transition from the first mode shape to the second.

In [ ]:
plt.plot(c_grid, np.abs(w1_ev2_num(Xasym, c_grid)/wabs_ev2_num(Xasym, c_grid)),'r', c_grid, np.abs(w2_ev2_num(Xasym, c_grid)/wabs_ev2_num(Xasym, c_grid)),'r--');
plt.xlabel('$c$');
plt.legend(['$w_1$', '$w_2$']);

For the higher buckling load it is the opposite transition than for the lower buckling load.

#### Visualization of the buckling shapes for different values of the spring stiffness

In [ ]:
c_selection = [1, 146, 300] 
for c_num in c_selection:
    x_grid = np.linspace(0, 1) 
    w_buckling_lower_load  = lambda x: w1_ev1_num(Xasym, c_num)/wabs_ev1_num(Xasym, c_num) * np.sin(np.pi*x) + w2_ev1_num(Xasym, c_num)/wabs_ev1_num(Xasym, c_num) * np.sin(2*np.pi*x) 
    w_buckling_higher_load = lambda x: w1_ev2_num(Xasym, c_num)/wabs_ev2_num(Xasym, c_num) * np.sin(np.pi*x) + w2_ev2_num(Xasym, c_num)/wabs_ev2_num(Xasym, c_num) * np.sin(2*np.pi*x) 
    plt.plot(x_grid, w_buckling_lower_load(x_grid),'b')
    plt.plot(x_grid, w_buckling_higher_load(x_grid),'r')
    plt.xlabel('x')
    plt.ylabel('w')
    plt.legend(['F_1', 'F_2'])
    plt.title('c=' + str(c_num))
    plt.xticks([0, 0.2, 0.4, 0.6, 0.8, 1, Xasym], ['0', '0.2', '0.4', '0.6', '0.8', '1', 'X'])
    plt.show(); 
    display(Math(f'w_l(X) = {sp.latex(w_buckling_lower_load(Xasym))}')) 
    display(Math(f'w_h(X) = {sp.latex(w_buckling_higher_load(Xasym))}'))